In [ ]:
import numpy as np
from math import ceil

from section2_1_b import get_f_prim
from section2_2 import get_E_T_star, get_E_H, get_E_C, get_E_W, get_E_L, get_E_V, get_E_M, get_heating_load, get_cooling_load, \
    get_virtual_heating_devices, get_virtual_heatsource, get_heating_mode, calc_L_HWH, get_E_E, get_E_G, get_E_K, \
    get_E_UT_H, calc_E_S, get_E_E_CG_h
from section2_3 import get_E_ST_star, calc_E_SH, calc_E_SC, calc_E_SV, calc_E_SW, calc_E_SL, calc_E_SM
from section3_1 import get_Q
from section3_2 import calc_insulation_performance
from section7_1_b import get_virtual_hotwater

# 第二章　単位住戸の一次エネルギー消費量 / Chapter 2 Designed Primary Energy Consumption

## 第一節　全般 / Section 1 General

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．住戸の床面積並びに主たる居室、その他の居室及び非居室の定義 / Floor area of dwelling unit and definition of main living room, other living room and non-living room

省略

Omission

### 6．電気の量 1kWh を熱量に換算する係数 / A factor to convert 1kWh of electricity into heat

省略

Omission

### 7．仮想居住人数 / Virtual number of people

省略

Omission

### 8．設計一次エネルギー消費量 / Design primary energy consumption

$$
\large 
E_{T} = E_{T}^* \times 10^{-3}
$$

In [ ]:
# ============================================================================
# 8. 設計一次エネルギー消費量
# ============================================================================

def calc_E_T(spec):
    E_H = None
    E_C = None
    E_W = None
    E_L = None
    E_V = None
    E_M = None
    E_gen = None
    E_E_gen = None
    E_S = None

    E_E = None
    E_G = None
    E_K = None
    UPL = None

    # ---- 事前データ読み込み ----

    # 日射量データの読み込み
    from section11_2 import load_solrad
    from section9_1 import get_E_E_PV_d_t

    solrad = None
    if (spec['SHC'] is not None or spec['PV'] is not None) and 'sol_region' in spec:
        if spec['sol_region'] is not None:
            solrad = load_solrad(spec['region'], spec['sol_region'])

    # ハイブリッド給湯機の仕様読み込み
    if spec['HW'] is not None and 'package_id' in spec['HW']:
        import section7_1_g_3
        section7_1_g_3.load_specification('HybridWaterHeater_package_list_TestCaseTest_sample.csv')

    # ---- 外皮の計算 ----

    # 外皮の断熱性能の計算
    if spec['ENV'] is not None:
        U_A, _, _, _, Q_dash, eta_H, eta_C, _ = calc_insulation_performance(**spec['ENV'])
        # 熱損失係数
        Q = get_Q(Q_dash)
    else:
        Q = None
        eta_H, eta_C = None, None

    # ---- 暖房設備 ----

    # 1 時間当たりの暖房設備の設計一次エネルギー消費量

    # 実質的な暖房機器の仕様を取得
    spec_MR, spec_OR = get_virtual_heating_devices(spec['region'], spec['H_MR'], spec['H_OR'])

    # 実質的な温水暖房機の仕様を取得
    spec_HS = get_virtual_heatsource(spec['region'], spec['H_HS'])

    # 暖房方式及び運転方法の区分
    mode_MR, mode_OR = get_heating_mode(region=spec['region'], H_MR=spec_MR, H_OR=spec_OR)

    # 暖房負荷の取得
    L_H_A_d_t, L_T_H_d_t_i, L_dash_H_R_d_t_i = get_heating_load(
        spec['region'], spec['sol_region'],
        spec['A_A'], spec['A_MR'], spec['A_OR'],
        Q, eta_H, spec['TS'], spec['r_A_ufvnt'], spec['HEX'],
        spec['underfloor_insulation'], spec['mode_H'],
        spec_MR, spec_OR, mode_MR, mode_OR, spec['SHC'])

    # 暖房日の計算
    if spec['SHC'] is not None and spec['SHC']['type'] == '空気集熱式':
        from section3_1_heatingday import get_heating_flag_d

        heating_flag_d = get_heating_flag_d(L_dash_H_R_d_t_i)
    else:
        heating_flag_d = None

    E_H = get_E_H(spec['region'], spec['sol_region'], spec['A_A'], spec['A_MR'], spec['A_OR'],
                  spec['mode_H'],
                  spec['H_A'], spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, spec['CG'], spec['SHC'],
                  heating_flag_d, L_H_A_d_t, L_T_H_d_t_i)

    # 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値
    UPL = get_E_UT_H(spec['region'], spec['A_A'], spec['A_MR'], spec['A_OR'], spec['mode_H'],
                     spec['H_A'], spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, spec['CG'],
                     L_H_A_d_t, L_T_H_d_t_i)
    UPL = np.sum(UPL)

    # 温水暖房負荷の計算
    L_HWH = calc_L_HWH(spec['A_A'], spec['A_MR'], spec['A_OR'], spec['HEX'], spec['H_HS'], spec['H_MR'],
                           spec['H_OR'], Q, spec['SHC'], spec['TS'], eta_H, spec['r_A_ufvnt'],
                           spec['region'], spec['sol_region'], spec['underfloor_insulation'],
                           spec['CG'])

    # ---- 冷房設備 ----

    # 冷房負荷の計算
    L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t = \
        get_cooling_load(spec['region'], spec['A_A'], spec['A_MR'], spec['A_OR'], Q, eta_C,
                         spec['NV_MR'], spec['NV_OR'], spec['r_A_ufvnt'], spec['underfloor_insulation'],
                         spec['mode_C'])

    # 1 年当たりの冷房設備の設計一次エネルギー消費量
    E_C = get_E_C(spec['region'], spec['A_A'], spec['A_MR'], spec['A_OR'],
                  spec['C_A'], spec['C_MR'], spec['C_OR'],
                  L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t)


    # ---- 給湯/コージェネ設備 ----

    # その他または設置しない場合
    spec_HW = get_virtual_hotwater(spec['region'], spec['HW'])

    # 1 年当たりの給湯設備（コージェネレーション設備を含む）の設計一次エネルギー消費量
    E_W, E_E_CG_gen_d_t, _, E_E_TU_aux_d_t, E_E_CG_h_d_t, E_G_CG_ded, e_BB_ave, Q_CG_h \
            = get_E_W(spec['A_A'], spec['region'], spec['sol_region'], spec_HW, spec['SHC'], spec['CG'],
                      spec['H_A'],
                      spec['H_MR'], spec['H_OR'], spec['H_HS'], spec['C_A'], spec['C_MR'], spec['C_OR'],
                      spec['V'],
                      spec['L'], spec['A_MR'], spec['A_OR'], Q, eta_H, eta_C,
                      spec['NV_MR'],
                      spec['NV_OR'], spec['TS'], spec['r_A_ufvnt'], spec['HEX'],
                      spec['underfloor_insulation'],
                      spec['mode_H'], spec['mode_C'])

    # ---- 照明,換気,その他設備 ----

    # 1 年当たりの照明設備の設計一次エネルギー消費量
    E_L = get_E_L(spec['A_A'], spec['A_MR'], spec['A_OR'], spec['L'])

    # 1 年当たりの機械換気設備の設計一次エネルギー消費量
    E_V = get_E_V(spec['A_A'], spec['V'], spec['HEX'])

    # 1年当たりのその他の設計一次エネルギー消費量
    E_M = get_E_M(spec['A_A'])

    # ---- 二次エネの計算 ----

    # 1 年当たりの設計消費電力量（kWh/年）
    E_E, E_E_PV_h_d_t, E_E_PV_d_t, E_E_CG_gen_d_t, E_E_CG_h_d_t, E_E_dmd_d_t, E_E_TU_aux_d_t = \
                get_E_E(spec['region'], spec['sol_region'], spec['A_A'], spec['A_MR'], spec['A_OR'],
                        spec_HW, Q, spec['TS'], eta_H, eta_C, spec['r_A_ufvnt'],
                        spec['underfloor_insulation'], spec['NV_MR'], spec['NV_OR'],
                        spec['mode_H'], spec['mode_C'],
                        spec['V'], spec['L'],
                        spec['H_A'], spec['H_MR'], spec['H_OR'], spec['H_HS'],
                        spec['CG'], spec['SHC'],
                        L_H_A_d_t, L_T_H_d_t_i,
                        spec['C_A'], spec['C_MR'], spec['C_OR'], L_CS_A_d_t, L_CL_A_d_t,
                        L_CS_d_t, L_CL_d_t,
                        spec['HEX'], spec['PV'], solrad
                        )
    f_prim = get_f_prim()
    E_gen = (np.sum(E_E_PV_d_t) + np.sum(E_E_CG_gen_d_t)) * f_prim / 1000

    # 1 年当たりの設計ガス消費量（MJ/年）
    E_G = get_E_G(spec['region'], spec['sol_region'], spec['A_A'], spec['A_MR'], spec['A_OR'],
                          Q, eta_H, eta_C, spec['NV_MR'], spec['NV_OR'], spec['TS'],
                          spec['r_A_ufvnt'], spec['HEX'], spec['underfloor_insulation'],
                          spec['H_A'], spec['H_MR'], spec['H_OR'], spec['H_HS'], spec['C_A'], spec['C_MR'],
                          spec['C_OR'], spec['V'], spec['L'], spec_HW, spec['SHC'],
                          spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, spec['mode_H'], spec['mode_C'],
                          spec['CG'],
                          L_T_H_d_t_i, L_HWH, heating_flag_d)

    # 1 年当たりの設計灯油消費量（MJ/年）
    E_K = get_E_K(spec['region'], spec['sol_region'], spec['A_A'], spec['A_MR'], spec['A_OR'],
                          spec['H_A'],
                          spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, spec['CG'],
                          L_T_H_d_t_i,
                          L_HWH, heating_flag_d, spec_HW, spec['SHC'])

    # ---- エネルギー利用効率化の評価 ----

    # エネルギー利用効率化設備による設計一次エネルギー消費量の削減量
    E_E_CG_h = get_E_E_CG_h(E_E_CG_h_d_t)
    E_S = calc_E_S(spec['region'], spec['sol_region'], spec['PV'], spec['CG'], E_E_dmd_d_t, E_E_CG_gen_d_t,
                   E_E_TU_aux_d_t, E_E_CG_h, E_G_CG_ded, e_BB_ave, Q_CG_h)

    E_E_gen = np.sum(get_E_E_PV_d_t(spec['PV'], solrad) + E_E_CG_gen_d_t)

    # ---- 合計 ----

    E_T = get_E_T(E_H, E_C, E_V, E_L, E_W, E_S, E_M)

    return E_T, E_H, E_C, E_V, E_L, E_W, E_S, E_M, UPL, E_gen, E_E_gen, E_E_PV_h_d_t, E_E, E_G, E_K


# 設計一次エネルギー消費量（GJ/年）(1)
def get_E_T(E_H, E_C, E_V, E_L, E_W, E_S, E_M):
    # 1 年当たりの設計一次エネルギー消費量（MJ/年）(s2-2-1)
    E_T_star = get_E_T_star(E_H, E_C, E_V, E_L, E_W, E_S, E_M)

    # 小数点以下一位未満の端数があるときはこれを切り上げる
    E_T = ceil(E_T_star / 100) / 10  # (1)

    return E_T

$E_{T}$：設計一次エネルギー消費量(GJ/年) / Design primary energy consumption  
$E_{T}^*$：設計一次エネルギー消費量(MJ/年) / Design primary energy consumption  

設計一次エネルギー消費量$E_{T}$は、数値に小数点以下一位未満の端数があるときは、これを切り上げる。 

Design Primary Energy Consumption $ E_{T} $ is rounded up if the number has a fraction less than one decimal place.

### 9．基準一次エネルギー消費量 / Standard primary energy consumption

$$
\large 
E_{ST} = E_{ST}^* \times 10^{-3}
$$

In [ ]:
# ============================================================================
# 9. 基準一次エネルギー消費量
# ============================================================================

def calc_E_ST(spec):
    # 1年当たりの機械換気設備の基準一次エネルギー消費量
    E_SV = calc_E_SV(spec['A_A'])

    # 1 時間当たりの暖房設備の設計一次エネルギー消費量
    E_SH = calc_E_SH(spec['type'], spec['tatekata'], spec['region'], spec['sol_region'], spec['A_A'],
                     spec['A_MR'], spec['A_OR'], spec['ENV'], spec['mode_H'], spec['H_MR'], spec['H_OR'])

    # 1 時間当たりの冷房設備の設計一次エネルギー消費量
    E_SC = calc_E_SC(spec['type'], spec['tatekata'], spec['region'], spec['A_A'], spec['A_MR'],
                     spec['A_OR'], spec['ENV'], spec['mode_C'])

    # 1年当たりの照明設備の基準一次エネルギー消費量
    E_SW = calc_E_SW(spec['region'], spec['A_A'], spec['HW'])

    # 1年当たりの照明設備の基準一次エネルギー消費量
    E_SL = calc_E_SL(spec['A_A'], spec['A_MR'], spec['A_OR'])

    # 1年当たりのその他の基準一次エネルギー消費量
    E_SM = calc_E_SM(spec['A_A'])

    # 合計
    E_ST = get_E_ST()

    return E_ST, E_SH, E_SC, E_SV, E_SL, E_SW, E_SM

# 基準一次エネルギー消費量（GJ/年）
def get_E_ST():
    # 小数点以下一位未満の端数があるときはこれを切り上げる
    return ceil(get_E_ST_star() / 100) / 10  # (2)

$E_{ST}$：基準一次エネルギー消費量(GJ/年) / Standard primary energy consumption  
$E_{ST}^*$：基準一次エネルギー消費量(MJ/年) / Standard primary energy consumption  

基準一次エネルギー消費量$E_{ST}$は、数値に小数点以下一位未満の端数があるときは、これを切り上げる。

The standard primary energy consumption $ E_ {ST} $ is rounded up if there are fractions less than one decimal place in the number.

### 10．BEI (Building Energy Index）

$$
\large 
BEI = \frac{E'_{T}}{E'_{ST}}
$$

$$
\large 
E'_{T} = (E^*_{T} - E_{M}) / 1000
$$

$$
\large 
E'_{ST} = (E^*_{ST} - E_{SM}) / 1000
$$

In [ ]:
# ============================================================================
# 10. BEI(Building Energy Index）
# ============================================================================

# BEI(Building Energy Index）
def get_BEI(E_T, E_M, E_ST, E_SM):
    f_prim = get_f_prim()
    # 小数点以下一位未満の端数があるときはこれを切り上げる
    E_T_dash = ceil((E_T_star - E_M) / 100) / 10  # (3-2)
    E_ST_dash = ceil((E_ST_star - E_SM) / 100) / 10  # (3-3)
    # 小数点以下二位未満の端数があるときはこれを切り上げる
    BEI = ceil(E_T_dash / E_ST_dash * 100) / 100  # (3-1)
    return BEI

$BEI$：Building Energy Index(-)  
$E_{ST}^*$：基準一次エネルギー消費量(MJ/年) / Standard primary energy consumption  
$E'_{ST}$：その他の基準一次エネルギー消費量を除いた基準一次エネルギー消費量(GJ/年) / Other criteria Primary energy consumption excluding primary energy consumption  
$E_{SM}$：1 年当たりのその他の基準一次エネルギー消費量(MJ/年) / Other baseline primary energy consumption per year  
$E_{T}^*$：設計一次エネルギー消費量(MJ/年) / Design primary energy consumption  
$E'_{T}$：その他の設計一次エネルギー消費量を除いた設計一次エネルギー消費量(GJ/年) / Other design primary energy consumption excluding primary energy consumption  
$E_{M}$：1 年当たりのその他の設計一次エネルギー消費量(MJ/年) / Other design primary energy consumption per year  

その他の基準一次エネルギー消費量を除いた基準一次エネルギー消費量$E'_{ST}$ およびその他の設計一次エネルギー消費量を除いた設計一次エネルギー消費量$E'_{T}$は、数値に小数点以下一位未満の端数があるときは、これを切り上げる。$BEI$は、数値に小数点以下二位未満の端数があるときは、これを切り上げる。

Other criteria Primary energy consumption $ E'_{ST} $ excluding primary energy consumption and design primary energy consumption $ E'_{T} $ excluding other design primary energy consumption are numerical values If there is a fraction less than one decimal place in, round it up. $ BEI $ rounds up if the number has a fraction less than two decimal places.

## 付録 A 「主たる居室」、「その他の居室」及び「非居室」の定義並びに各床面積及び床面積の合計の算出方法 / Appendix A Definitions of "main room", "other rooms" and "non-rooms" and calculation method of total floor area and total floor area

省略

Omission

## 付録 B 電気の量 1kWh を熱量に換算する係数 / Appendix B  A factor to convert 1kWh of electricity into heat

電気の量1kWhを熱量に換算する係数は、1キロワット時につき 9760キロジュールを使用するものとする。

The factor for converting 1 kWh of electricity into heat is assumed to be 9,760 kilojoules per kilowatt hour.

In [ ]:
# 電気の量 1kWh を熱量に換算する係数 (kJ/kWh)
def get_f_prim():
    return 9760

## 付録 C 仮想居住人数 / Appendix C Virtual number of people

$$
\large
n_{p} = \left\{
\begin{array}{ll}
1.0 & (A_A < 30)\\
A_A \div 30 & (30 \leq A_A < 120)\\
4.0 & (120 \leq A_A)
\end{array}
\right.
$$

In [ ]:
def get_n_p(A_A):
    if A_A < 30:
        return 1.0
    elif A_A < 120:
        return A_A / 30
    else:
        return 4.0

$n_{p}$：仮想居住人数 / Virtual number of people  
$A_A$：床面積の合計(m<sup>2</sup>) / Total floor area  